# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [18]:
import os
import requests
import pandas as pd

#Import CitiBike.csv.
df = pd.read_csv("../data/vancouver_citibike_stations.csv")

citibike_ids = [] #Stored and used as a variable to join DataFrames.

#Loop to pass CitiBike coordinates to Foursquare URL.
for index, row in df.iterrows():
    lat = row['latitude']
    long = row['longitude']
    bikeid = row['citibike_id']
    citibike_ids.append(bikeid)

df_citibike_id = pd.DataFrame({'citibike_id': citibike_ids})

#Foursquare API key in an environment variable.
fsq_api_key = os.getenv("FOURSQUARE_API_KEY")

#Function to pass information to Foursquare API:
def get_venues_fsq(latitude, longitude, radius, api_key):
    url = f"https://api.foursquare.com/v3/places/search?ll={latitude},{longitude}&radius={radius}"
    headers = {"Accept": "application/json", "Authorization": fsq_api_key}
    response = requests.get(url, headers=headers)
    return response

#Get results for locations in Vancouver.
vancouver_location = get_venues_fsq(
    latitude = lat,
    longitude = long,
    radius = 1000,
    api_key = fsq_api_key
)

data = vancouver_location.json()

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [19]:
#Columns were cleaned by dropping irrelevant columns and renaming column names.
vancouver_fsq = pd.json_normalize(data['results'])

vancouver_fsq_col = vancouver_fsq[['fsq_id', 'distance', 'name', 'geocodes.main.latitude', 'geocodes.main.longitude', 'location.formatted_address', 'location.locality','location.region','location.postcode']]

vancouver_fsq_rename_col = vancouver_fsq_col.rename(columns={
    'name': 'fsq_name',
    'distance': 'fsq_distance',
    'geocodes.main.latitude': 'latitude',
    'geocodes.main.longitude': 'longitude',
    'location.formatted_address': 'fsq_address',
    'location.locality': 'fsq_city',
    'location.region': 'fsq_province',
    'location.postcode': 'fsq_postalcode'
    }
)

vancouver_fsq_rename_col['citibike_id'] = df_citibike_id['citibike_id']


Put your parsed results into a DataFrame

In [21]:
vancouver_fsq_rename_col.head(5)

#Save As .csv file - vancouver_fsq_rename_col.to_csv("../data/vancouver_fsq_locations.csv")

,fsq_id,fsq_distance,fsq_name,latitude,longitude,fsq_address,fsq_city,fsq_province,fsq_postalcode,citibike_id
0,50209a59e4b072cf35a19131,103,Menchies,49.254394,-123.234947,"3358 Wesbrook Mall (at UBC), Vancouver BC V6S 0A8",Vancouver,BC,V6S 0A8,7a19c49f486d7c0c02b3685d7b240448
1,4ac93fa4f964a5205dbf20e3,21,Carlton Cards SAVE on FOODS,49.254768,-123.236681,"5945 Berton Ave (at UBC Wesbrook Village), Van...",Vancouver,BC,V6S 0B3,32603a87cfca71d0f7dfa3513bad69d5
2,5c35d16e4aa3f8002c929edd,25,Starbucks,49.254768,-123.236681,"5945 Berton Ave, Vancouver BC V6S 0B3",Vancouver,BC,V6S 0B3,6d42fa40360f9a6b2bf641c7b8bb2862
3,503d559ae4b04ada6683794c,70,Wesbrook Village,49.254501,-123.235605,,NaN,NaN,NaN,66f873d641d448bd1572ab086665a458
4,5481eefe498e3a15c818d155,38,Doughgirls Bakeshop,49.254542,-123.235770,"3322 Shrum Lane, Vancouver BC V6S 0B1",Vancouver,BC,V6S 0B1,485d4d24c803cfde829ab89699fed833


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [22]:
import requests
import os
import pandas as pd

#Import CitiBike.csv.
df = pd.read_csv("../data/vancouver_citibike_stations.csv")

#Stored and used as variables to join DataFrames.
citibike_ids = []
latitudes = []
longitudes = []

#Loop to pass CitiBike coordinates to Yelp URL.
for index, row in df.iterrows():
    lat = row['latitude']
    latitudes.append(lat)
    long = row['longitude']
    longitudes.append(lat)
    bikeid = row['citibike_id']
    citibike_ids.append(bikeid)

df_citibike_data = pd.DataFrame({'latitude': latitudes, 'longitude': longitudes, 'citibike_id': citibike_ids})

#Yelp API key in an environment variable.
yelp_api_key = os.getenv("YELP_API_KEY")

#Function to pass information to Foursquare API:
def get_venues_yelp(latitude, longitude, radius, api_key):
    url = f"https://api.yelp.com/v3/businesses/search?latitude={latitude}&longitude={longitude}&radius={radius}"
    headers = {"Accept": "application/json", "Authorization": f"Bearer {api_key}"}
    response = requests.get(url, headers=headers)
    return response

vancouver_location = get_venues_yelp(
    latitude = lat, 
    longitude = long, 
    radius = 1000,
    api_key = yelp_api_key
)

data = vancouver_location.json()

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [23]:
data = vancouver_location.json()

#Columns were cleaned by dropping irrelevant columns and renaming column names.
vancouver_yelp = pd.json_normalize(data['businesses'])

vancouver_yelp_col = vancouver_yelp[['id', 'name', 'review_count', 'rating', 'location.display_address']]

vancouver_yelp_rename_col = vancouver_yelp_col.rename(columns={
    'id': 'yelp_id',
    'name': 'yelp_name',
    'review_count': 'yelp_review_count',
    'rating': 'yelp_rating',
    'location.display_address': 'yelp_address'
    }
)

vancouver_yelp_rename_col_add = pd.concat([vancouver_yelp_rename_col, df_citibike_data], axis=1)

Put your parsed results into a DataFrame

In [27]:
vancouver_yelp_rename_col_add.head(5)

#Save As .csv file - vancouver_yelp_rename_col_add.to_csv("../data/vancouver_yelp_locations.csv")

,yelp_id,yelp_name,yelp_review_count,yelp_rating,yelp_address,latitude,longitude,citibike_id
0,FPP0VvrPrV8RrF6Yye31ig,Chef Hung Taiwanese Beef Noodle,67.0,2.7,"[3313 Shrum Lane, Unit 102, Vancouver, BC V6S ...",49.262487,49.262487,7a19c49f486d7c0c02b3685d7b240448
1,oSRxJSXssqOAAVlF1DS0Zg,Doughgirls,63.0,4.4,"[3322 Shrum Lane, Vancouver, BC V6S 0B9, Canada]",49.274566,49.274566,32603a87cfca71d0f7dfa3513bad69d5
2,uW6NtwN8hxrOLMyPi9IVcA,Neptune Chinese Kitchen,37.0,2.9,"[3337 Shrum Lane, Vancouver, BC V6S 0C8, Canada]",49.279764,49.279764,6d42fa40360f9a6b2bf641c7b8bb2862
3,estK0IAUPgS0bftyYHTXNA,Blenz Coffee,29.0,3.3,"[3381 Wesbrook Mall, Vancouver, BC V6S 0B1, Ca...",49.260599,49.260599,66f873d641d448bd1572ab086665a458
4,W3fcG0C3w-6ARo_5yO4FYQ,Togo Sushi,30.0,2.3,"[3380 Shrum Lane, Vancouver, BC V6T 1W5, Canada]",49.264215,49.264215,485d4d24c803cfde829ab89699fed833


# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

The Yelp API provided more completed data.  Not only did it generate location details, but also, there were additional data on ratings and review counts, which could be variables used later in the statistical modeling portion of the project.

Get the top 10 restaurants according to their rating

In [6]:
import pandas as pd

df = pd.read_csv("../data/vancouver_yelp_locations.csv")

top_10_yelp = df.sort_values(by='yelp_rating', ascending=False)

top_10_yelp[['yelp_name', 'yelp_rating']].head(10)

,yelp_name,yelp_rating
19,Sharetea,5.0
13,Spanish Banks Beach Park,4.7
9,Bombay Masala Indian Restaurant,4.6
6,Museum of Anthropology,4.5
2,Doughgirls,4.4
5,Rice Burger,4.4
12,UBC Botanical Garden,4.3
18,Great Dane Coffee,4.1
7,Jamjar Canteen,4.0
3,Burgoo Bistro,3.9
